In [1]:
import dask.dataframe as dd
import pandas as pd
from dask.diagnostics import ProgressBar
import dask

dask.config.set({
    "scheduler": "threads",
    "dataframe.shuffle.method": "tasks"
})


In [3]:
loan_dd = dd.read_csv(
    r"E:\mergeloans\*.csv",
    sep=";",
    dtype=str,
    assume_missing=True
)

refund_dd = dd.read_csv(
    r"E:\mergerefunds\*.csv",
    sep=";",
    dtype=str,
    assume_missing=True
)

print("Loans partitions:", loan_dd.npartitions)
print("Refund partitions:", refund_dd.npartitions)


Loans partitions: 16
Refund partitions: 19


In [4]:
def clean_client_id(x):
    if pd.isna(x):
        return ""
    x = "".join(ch for ch in str(x) if ch.isdigit())
    return x[-9:].zfill(9)

loan_dd["clientID"] = loan_dd["clientID"].map(clean_client_id, meta=("clientID", "string"))
loan_dd["loanID"] = loan_dd["loanID"].astype("string")

refund_dd["clientID"] = refund_dd["clientID"].map(clean_client_id, meta=("clientID", "string"))
refund_dd["loanID"] = refund_dd["loanID"].astype("string")

loan_dd["loanDate"] = dd.to_datetime(loan_dd["loanDate"], dayfirst=True, errors="coerce")
loan_dd["dueDate"] = dd.to_datetime(loan_dd["dueDate"], dayfirst=True, errors="coerce")
refund_dd["date"] = dd.to_datetime(refund_dd["date"], dayfirst=True, errors="coerce")

print("ID and date cleaning done")


ID and date cleaning done


In [4]:
with ProgressBar():
    refund_last = (
        refund_dd
        .groupby("loanID")
        .agg({"date": "max"})
        .reset_index()
        .rename(columns={"date": "last_refund_date"})
    )

refund_last.head()


C:\Users\Pro\anaconda3\Lib\site-packages\dask\dataframe\dask_expr\_expr.py:1543: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return get_meta_library(args[0]).to_datetime(*args, **kwargs)
C:\Users\Pro\anaconda3\Lib\site-packages\dask\dataframe\dask_expr\_expr.py:1543: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return get_meta_library(args[0]).to_datetime(*args, **kwargs)
C:\Users\Pro\anaconda3\Lib\site-packages\dask\dataframe\dask_expr\_expr.py:1543: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return get_meta_library(args[0]).to_datetime(*args, **kwargs)
C:\Users\Pro\

,loanID,last_refund_date
0,RFS17X01T20190820160342MU,2019-08-21
1,RFS17X01T20190821043212MU,2019-08-21
2,RFS17X01T20190822172742MU,2019-08-22
3,RFS17X01T20190822082353MU,2019-08-22
4,RFS17X01T20190823084258MU,2019-08-23


C:\Users\Pro\anaconda3\Lib\site-packages\dask\dataframe\dask_expr\_expr.py:1543: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return get_meta_library(args[0]).to_datetime(*args, **kwargs)
C:\Users\Pro\anaconda3\Lib\site-packages\dask\dataframe\dask_expr\_expr.py:1543: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return get_meta_library(args[0]).to_datetime(*args, **kwargs)
C:\Users\Pro\anaconda3\Lib\site-packages\dask\dataframe\dask_expr\_expr.py:1543: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return get_meta_library(args[0]).to_datetime(*args, **kwargs)
C:\Users\Pro\

In [5]:
loan_dd = loan_dd.merge(
    refund_last,
    on="loanID",
    how="left"
)

loan_dd.head()


,clientID,loanID,loanAmount,amountwithInterest,amountPaid,dueAmount,totalDueAmount,loanDate,dueDate,carryoverDate,...,loanStatus,is_complete,isCarryover,isReconducted,isPenalty,penaltyDate,penaltyAmount,penaltyBalance,dateTime,last_refund_date
0,678445768,RFS17X01T20190820160342MU,4000.0,4300.0,4200.0,0.0,0.0,2019-08-20,2019-09-04,19/09/2019,...,isBonusPeriod,True,False,False,False,<NA>,5000.0,0.0,20/08/2019 16:03,2019-08-21
1,650535634,RFS17X01T20190821043212MU,500.0,538.0,525.0,0.0,0.0,2019-08-21,2019-09-05,20/09/2019,...,isBonusPeriod,True,False,False,False,<NA>,5000.0,0.0,21/08/2019 04:32,2019-08-21
2,674644740,RFS17X01T20190827111048MU,5000.0,5375.0,5375.0,0.0,0.0,2019-08-27,2019-09-11,26/09/2019,...,Normal,True,False,False,False,<NA>,5000.0,0.0,27/08/2019 11:10,2019-09-09
3,677186102,NA05X01T20191203092607MU,10000.0,11500.0,11500.0,0.0,0.0,2019-12-03,2020-01-02,02/01/2020,...,isCarryOver,True,False,False,False,<NA>,5000.0,0.0,03/12/2019 09:26,2020-06-05
4,678212131,NA05X01T20191206163633MU,5000.0,5375.0,5779.0,0.0,0.0,2019-12-06,2019-12-21,05/01/2020,...,isCarryOver,True,False,False,False,<NA>,5000.0,0.0,06/12/2019 16:36,2021-02-01


In [6]:
loan_dd["payment_delta_days"] = (
    loan_dd["dueDate"] - loan_dd["last_refund_date"]
).dt.days

loan_dd[["clientID", "loanID", "payment_delta_days"]].head()


,clientID,loanID,payment_delta_days
0,678445768,RFS17X01T20190820160342MU,14
1,650535634,RFS17X01T20190821043212MU,15
2,674644740,RFS17X01T20190827111048MU,2
3,677186102,NA05X01T20191203092607MU,-155
4,678212131,NA05X01T20191206163633MU,-408


In [12]:
with ProgressBar():
    loan_dd.compute().to_csv(
        r"E:\merge_loan_refund_base.csv",
        index=False,
        sep=";"
    )

print("New merge has been saved")


[########################################] | 100% Completed | 37m 41s
New merge has been saved


In [11]:
from dask.diagnostics import ProgressBar

with ProgressBar():
    # Compute into pandas, then take first 10 rows and 10 columns
    sample = loan_dd.head(10)

    # Save sample to CSV
    sample.to_csv(
        r"E:\merge_loan_refund_sample.csv",
        index=False,
        sep=";"
    )

print("Sample of 10 rows has been saved")


[########################################] | 100% Completed | 150.66 ms
Sample of 10 rows and 10 columns has been saved
